In [1]:
import os
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk


In [2]:
os.getcwd()

'C:\\Users\\LENOVO'

In [3]:
#changing the directory 
os.chdir("E:\\INSOFE\\text mining\\movie_reviews_sentiment_analysis")

In [4]:
#reading the data
train = pd.read_csv("labeledTrainData.tsv", header = 0, delimiter="\t",quoting=3)
test  = pd.read_csv("testData.tsv",header = 0,delimiter="\t",quoting = 3)
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0,delimiter="\t", quoting=3 )

In [12]:
train.shape
#test.shape

(25000, 3)

In [6]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [5]:
# let's make all these preprocessing steps combine into a function

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [6]:
# Download the punkt tokenizer for sentence splitting
#import nltk.data
#nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [7]:
#Now we can apply this function to prepare our data for input to Word2Vec
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
E:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
E:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup

Parsing sentences from unlabeled set


E:\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
E:\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
E:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
E:\Anaconda3\lib\site-packages\bs4\__init__.py:219: U

In [8]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [40]:
len(sentences)

795538

In [9]:
from gensim.models import Word2Vec

E:\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
import gensim

In [11]:
print(gensim)

<module 'gensim' from 'E:\\Anaconda3\\lib\\site-packages\\gensim\\__init__.py'>


In [12]:
model = Word2Vec(sentences,workers = 4,size = 300,min_count=10)

In [13]:
model_name = "word2vec_model"
model.save(model_name)

In [14]:
#check the effeciency of word2vec model created
print(model.doesnt_match("man woman child kitchen".split()))
print(model.doesnt_match("france england germany berlin".split()))#berlin is a not a country
print(model.doesnt_match("paris berlin london austria".split())) #austria is not a city

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  


kitchen
berlin
paris


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  after removing the cwd from sys.path.


In [50]:
model.most_similar("man")

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6373317241668701),
 ('lady', 0.6072850227355957),
 ('lad', 0.5571324825286865),
 ('person', 0.5521164536476135),
 ('guy', 0.5507382154464722),
 ('monk', 0.5397407412528992),
 ('boy', 0.5277280807495117),
 ('men', 0.5250078439712524),
 ('chap', 0.5238170623779297),
 ('soldier', 0.5158064961433411)]

In [53]:
model.most_similar('bollywood')

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hindi', 0.7035006284713745),
 ('hollywood', 0.6899725794792175),
 ('sf', 0.6346628665924072),
 ('hk', 0.6173146963119507),
 ('mainstream', 0.5975834131240845),
 ('filipino', 0.5743932723999023),
 ('genre', 0.5669865608215332),
 ('blaxploitation', 0.554599404335022),
 ('tamil', 0.5512004494667053),
 ('disney', 0.5451921820640564)]

In [54]:
 model.most_similar("awful")

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7806997299194336),
 ('horrible', 0.7684855461120605),
 ('atrocious', 0.7654402256011963),
 ('abysmal', 0.7346374988555908),
 ('horrendous', 0.7188792824745178),
 ('appalling', 0.7181710004806519),
 ('dreadful', 0.7067798376083374),
 ('horrid', 0.7040719389915466),
 ('laughable', 0.6457450985908508),
 ('bad', 0.6269937753677368)]

In [15]:
model = Word2Vec.load("word2vec_model")

In [16]:
model.syn1neg.shape

(34119, 300)

In [18]:
model["flower"].shape

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

### From Words To Paragraphs: Vector Averaging

In [32]:
import numpy as np  

num_features = 300
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec




In [39]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="int32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print( "Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [40]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model,num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features)

E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 25000


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

notes on word2vec updations

In order to share word vector querying code between different training algos(Word2Vec, Fastext, WordRank, VarEmbed) the authors have separated storage and querying of word vectors into a separate class KeyedVectors.

Two methods and several attributes in word2vec class have been deprecated.

Methods

load_word2vec_format
save_word2vec_format
Attributes

syn0norm
syn0
vocab
index2word
These have been moved to KeyedVectors class.

After upgrading to this release you might get exceptions about deprecated methods or missing attributes.

To remove the exceptions, you should use

KeyedVectors.load_word2vec_format (instead ofWord2Vec.load_word2vec_format)

1.word2vec_model.wv.save_word2vec_format (instead of  word2vec_model.save_word2vec_format)

2.model.wv.syn0norm instead of  (model.syn0norm)

3.model.wv.syn0 instead of  (model.syn0)

4.model.wv.vocab instead of (model.vocab)

5.model.wv.index2word instead of (model.index2word)

In [15]:
test_features = vect.fit_transform(processed_test_reviews)
test_features = test_features.toarray()

In [16]:
print(test_features.shape)
test_features

(25000, 5000)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
#look into the vocabulary
vocab = vect.get_feature_names()
vocab

['aaron',
 'abandoned',
 'abc',
 'abilities',
 'ability',
 'able',
 'aboard',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abused',
 'abysmal',
 'academy',
 'accent',
 'accents',
 'accept',
 'acceptable',
 'accepted',
 'accident',
 'accidentally',
 'accompanied',
 'accomplished',
 'according',
 'account',
 'accurate',
 'accurately',
 'accused',
 'achieve',
 'achieved',
 'achievement',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'actions',
 'active',
 'activities',
 'actor',
 'actors',
 'actress',
 'actresses',
 'acts',
 'actual',
 'actually',
 'ad',
 'adam',
 'adams',
 'adaptation',
 'adaptations',
 'adapted',
 'add',
 'added',
 'addict',
 'addiction',
 'adding',
 'addition',
 'adds',
 'adequate',
 'admire',
 'admit',
 'admittedly',
 'adorable',
 'adult',
 'adults',
 'advance',
 'advantage',
 'adventure',
 'adventures',
 'advertising',
 'advice',
 'advise',
 'affair',
 'affairs',
 'affected',
 'affection',
 'afford',
 'afghanistan',
 'aforementioned

In [18]:
# to count each word in the vocabulary
cnt = np.sum(train_features,axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, cnt):
    print (count, tag)

187 aaron
125 abandoned
108 abc
454 abilities
1259 ability
85 able
116 aboard
83 absence
352 absent
1485 absolute
306 absolutely
192 absurd
91 abuse
98 abused
297 abysmal
485 academy
203 accent
300 accents
130 accept
144 acceptable
92 accepted
318 accident
200 accidentally
88 accompanied
124 accomplished
296 according
186 account
81 accurate
284 accurately
123 accused
179 achieve
139 achieved
124 achievement
90 across
971 act
1251 acted
658 acting
6490 action
3354 actions
311 active
83 activities
2389 actor
4486 actors
1219 actress
369 actresses
394 acts
793 actual
4237 actually
148 ad
302 adam
98 adams
453 adaptation
80 adaptations
154 adapted
810 add
439 added
166 addict
347 addiction
337 adding
113 addition
124 adds
621 adequate
134 admire
101 admit
510 admittedly
376 adorable
100 adult
90 adults
153 advance
510 advantage
204 adventure
91 adventures
259 advertising
90 advice
346 advise
93 affair
113 affairs
104 affected
126 affection
343 afford
212 afghanistan
255 aforementioned
187

82 dee
653 deep
177 deeper
320 deeply
107 defeat
95 defend
86 defense
87 defined
117 definite
1580 definitely
83 definition
204 degree
97 deliberately
108 delicate
157 delight
274 delightful
328 deliver
243 delivered
94 delivering
356 delivers
184 delivery
90 demand
113 demands
89 demon
79 demons
183 demonstrates
192 dennis
82 department
176 depicted
152 depicting
141 depiction
194 depicts
210 depressed
82 depressing
172 depression
90 depth
99 deranged
226 derivative
129 descent
511 describe
86 described
151 describes
89 describing
330 description
233 desert
102 deserve
175 deserved
250 deserves
287 design
291 designed
591 desire
342 desired
206 desires
91 despair
304 desperate
91 desperately
86 despite
323 destiny
179 destroy
104 destroyed
1364 destruction
104 detail
219 detailed
178 details
94 detective
142 determined
345 develop
108 developed
410 developing
460 development
164 develops
252 device
404 devil
102 devito
641 devoid
144 devoted
181 dexter
314 dialog
104 dialogs
109 dialo

1111 heads
231 health
1328 hear
225 heard
135 hearing
128 heart
320 heartbreaking
180 hearted
492 hearts
222 heat
85 heaven
391 heavily
152 heavy
97 heck
1025 heist
90 held
1895 helen
324 helicopter
176 hell
360 help
155 helped
407 helping
1056 helps
318 hence
115 henry
291 hercules
136 hero
409 heroes
342 heroic
210 heroin
103 heroine
144 herzog
2161 hey
289 hi
106 hidden
202 hide
125 hideous
1147 hiding
973 high
86 higher
243 highest
152 highlight
147 highlights
103 highly
181 hilarious
84 hilariously
130 hill
188 hills
407 hilton
86 hindi
1332 hint
1088 hints
209 hip
305 hippie
272 hire
137 hired
126 historical
188 historically
545 history
209 hit
300 hitch
167 hitchcock
367 hitler
148 hits
113 hitting
107 ho
1907 hoffman
163 hogan
113 hokey
133 hold
1877 holding
140 holds
103 hole
90 holes
481 holiday
453 hollow
99 holly
191 hollywood
173 holmes
162 holocaust
99 holy
139 homage
95 home
1447 homosexual
144 honest
214 honestly
82 honesty
273 hong
407 honor
98 hood
134 hook
1201 hooke

287 mixed
103 mixture
83 miyazaki
83 mm
157 mob
240 mode
105 model
929 models
122 modern
89 modest
367 mom
1112 moment
1663 moments
84 money
2362 monkees
85 monkey
132 monster
98 monsters
655 montage
277 montgomery
103 month
95 months
148 monty
272 mood
432 moody
98 moon
296 moore
227 moral
366 morality
122 morals
275 moreover
266 morgan
83 morning
98 moron
941 morris
1524 moses
449 mostly
122 mother
95 motion
103 motivation
196 motivations
110 motives
168 motorcycle
332 mountain
727 mountains
322 mouse
206 mouth
115 move
530 moved
44031 movement
7663 movements
854 moves
1448 movie
260 movies
350 moving
180 mr
86 mrs
9765 ms
140 mst
190 mtv
108 much
86 muddled
1063 multi
260 multiple
179 mummy
109 mundane
367 muppet
213 murder
81 murdered
96 murderer
3056 murdering
992 murderous
175 murders
85 murphy
3249 murray
97 museum
105 music
404 musical
850 musicals
98 musician
218 musicians
437 must
1604 mutant
799 mute
96 myers
391 mysterious
227 mystery
192 myth
424 nail
125 naive
342 naked
8

126 regard
213 regarded
89 regarding
807 regardless
986 regards
132 region
242 regret
106 regular
238 relate
310 related
209 relation
78 relations
120 relationship
439 relationships
583 relative
309 relatively
105 relatives
91 release
1702 released
258 releases
157 relevant
347 relief
297 relies
175 religion
163 religious
189 rely
108 remain
90 remained
719 remaining
214 remains
337 remake
177 remarkable
143 remarkably
204 remember
119 remembered
123 remind
162 reminded
96 reminds
213 reminiscent
103 remote
99 remotely
132 remove
191 removed
190 rendered
132 rendition
231 rent
222 rental
107 rented
80 renting
113 repeat
85 repeated
84 repeatedly
130 repeating
89 repetitive
89 replace
499 replaced
83 report
79 reporter
122 represent
88 representation
275 represents
1803 reputation
119 require
90 required
632 requires
82 rescue
274 research
161 resemblance
82 resembles
624 resist
117 resolution
126 resort
307 respect
151 respectable
189 respected
256 response
122 responsibility
183 respo

572 suspenseful
83 suspicious
97 sutherland
97 swayze
195 swear
114 swedish
229 sweet
199 swimming
111 switch
370 sword
181 sydney
79 symbolic
97 symbolism
99 sympathetic
3507 sympathy
986 syndrome
2192 synopsis
955 system
790 table
933 tad
586 tag
268 take
166 taken
842 takes
126 taking
954 tale
220 talent
121 talented
111 talents
80 tales
115 talk
234 talked
82 talking
211 talks
291 tall
174 tame
437 tank
95 tape
91 tarantino
315 target
136 tarzan
137 task
314 taste
82 tastes
823 taught
138 taylor
324 tea
82 teach
305 teacher
195 teachers
80 teaches
150 teaching
136 team
245 teams
198 tear
218 tears
339 tech
325 technical
221 technically
217 technique
198 techniques
181 technology
904 ted
1718 tedious
613 teen
880 teenage
122 teenager
829 teenagers
215 teens
101 teeth
86 telephone
152 television
541 tell
167 telling
433 tells
1638 temple
274 ten
433 tend
142 tender
123 tends
197 tense
103 tension
89 term
112 terms
236 terrible
83 terribly
210 terrific
158 terrifying
745 territory
439

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [61]:
model_randomforest = RandomForestClassifier(n_estimators= 100)

In [62]:
model_fit = model_randomforest.fit(train_features,train["sentiment"])

NameError: name 'train_features' is not defined

In [23]:
pred_test = model_fit.predict(test_features)

In [24]:
#copy it to dataframe
output = pd.DataFrame(data={"id":test["id"], "sentiment":pred_test})
#Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )